# Agentes

**Instalações e importações necessárias**


In [ ]:
!pip install -qU langchain langchain-huggingface langchain_community langchainhub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.tools import tool
from transformers import pipeline
from langchain import hub
import getpass
import os

API key para usar endpoints do HuggingFace, através do LangChain

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_api_key"

Escolha do modelo a ser utilizado:

In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.5, verbose=True
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**Criação de tools**

Tool para resumir textos:

In [ ]:
summarization_pipeline = pipeline("summarization")

@tool
def summarize_text(text: str) -> str:
  """Summarize the provided text."""

  summary = summarization_pipeline(text, max_length=20, min_length=10, do_sample=False)
  return summary[0]['summary_text']

Tool para análise de sentimentos:

In [ ]:
sentiment_pipeline = pipeline('sentiment-analysis')

@tool
def analyze_sentiment(text: str) -> dict:
    """Analyze sentiment of the provided text."""
    return sentiment_pipeline(text)[0]

Tool para identificar emoções:

In [ ]:
emotion_pipeline = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

@tool
def classify_emotion(text: str) -> dict:
    """Classify the emotion expressed in the provided text."""
    return emotion_pipeline(text)[0]

Tool para identificar tópicos em um texto:

In [ ]:
topic_classification_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

@tool
def identify_topic(text: str) -> dict:
    """Identify the main topic in the provided text using zero-shot classification."""
    candidate_labels = ["technology", "health", "finance", "sports", "education", "entertainment", "politics"]
    result = topic_classification_pipeline(text, candidate_labels)
    return {
        "label": result['labels'][0],
        "score": result['scores'][0]
    }

**Criação do agente**

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

tools = [analyze_sentiment, summarize_text, classify_emotion, identify_topic]

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

**Exemplos de funcionamento**

Entrada requer análise de sentimentos


In [ ]:
result = agent_executor.invoke({"input": "What is the sentiment of the phrase: 'That was a fun movie!'?"})



> Entering new AgentExecutor chain...
 I need to analyze the sentiment of the given phrase
Action: analyze_sentiment
Action Input: 'That was a fun movie!'
Observation{'label': 'POSITIVE', 'score': 0.9996129870414734} The sentiment of the phrase is positive
Final Answer: The sentiment of the phrase 'That was a fun movie!' is positive.</s>

> Finished chain.


Entrada requer sumarização de textos

In [ ]:
result = agent_executor.invoke({"input": "Make the follow text shorter: 'Interstellar is an epic journey that transcends the bounds of space and time. Directed by Christopher Nolan, the film immerses viewers in a dystopian future where Earth faces a devastating environmental crisis. Amidst this turmoil, Cooper, a former pilot turned farmer, is recruited to lead an interstellar expedition in search of a new home for humanity.’ "})



> Entering new AgentExecutor chain...
 I need to summarize the text
Action: summarize_text
Action Input: 'Interstellar is an epic journey directed by Christopher Nolan in a dystopian future where Earth faces an environmental crisis. Cooper, a former pilot, is recruited for an interstellar expedition.'
Observation Interstellar is an epic journey directed by Christopher Nolan in a dystopian future where Earth faces an The summarize_text function returned the first part of the summary. I need to get the second part.
Action: summarize_text
Action Input: 'Cooper, a former pilot, is recruited for an interstellar expedition.'
Observation Cooper, a former pilot, is recruited for an interstellar expedition . 'Cooper, The summarize_text function returned the second part of the summary. I have the final answer.
Final Answer: Interstellar is an epic journey directed by Christopher Nolan in a dystopian future where Earth faces an environmental crisis. Cooper, a former pilot, is recruited for an i

Entrada requer identificação de emoções


In [ ]:
result = agent_executor.invoke({"input": "Classify the emotion expressed by the phrase: 'I did not have fun watching the movie!'?"})



> Entering new AgentExecutor chain...
 I need to classify the emotion expressed in the given text
Action: classify_emotion
Action Input: 'I did not have fun watching the movie!'
Observation{'label': 'anger', 'score': 0.8137744665145874} I now know the final answer
Final Answer: The person expressed anger while watching the movie.</s>

> Finished chain.


Entrada requer identificação de tópicos

In [ ]:
result = agent_executor.invoke({"input": "What are the topics on the following text: 'Streaming services offer a range of digital content like movies, TV shows, music, and podcasts for on-demand consumption, typically through subscription or pay-per-use models.’?"})



> Entering new AgentExecutor chain...
 I need to identify the topics in the text
Action: identify_topic
Action Input: 'Streaming services offer a range of digital content like movies, TV shows, music, and podcasts for on-demand consumption, typically through subscription or pay-per-use models.'
Observation{'label': 'technology', 'score': 0.5736660957336426} I only got one topic from the text, I should check if there's more information
Action: summarize_text
Action Input: 'Streaming services offer a range of digital content like movies, TV shows, music, and podcasts for on-demand consumption, typically through subscription or pay-per-use models.'
Observation Streaming services offer a range of digital content like movies, TV shows, music, and It seems like the summary didn't capture the full text, let me try again
Action: summarize_text
Action Input: 'Streaming services offer a range of digital content like movies, TV shows, music, and podcasts for on-demand consumption, typically thr

# LangSmith

Instalações e importações necessárias

In [ ]:
!pip install -qU langsmith langchain-cohere
from langchain_cohere import ChatCohere

API keys necessárias

In [ ]:
os.environ["COHERE_API_KEY"] = "your_api_key"

In [ ]:
langchain_api_key = "your_api_key"

Instanciação do modelo

In [ ]:
chat = ChatCohere()

Definição das variáveis de ambiente para conexão com o LangSmith

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangChain_Presentation"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key

In [ ]:
from langsmith import Client
client = Client()

Exemplos que serão inseridos em um dataset

In [ ]:
example_inputs = [
    ("What is the largest mammal?", "The blue whale"),
    ("What do mammals and birds have in common?", "They are both warm-blooded"),
    ("What are reptilees knwown for?", "Having scales"),
    ("What's the main characteristic of amphibians?", "They live both in water and on land"),
]

Criação de um dataset para as perguntas e respostas

In [ ]:
dataset = client.create_dataset(
    dataset_name="Elementary Animal Questions",
    description="Questions and answers about animal phylogenetics."
)

Inserção dos exemplos no dataset criados

In [ ]:
for input_prompt, output_answer in example_inputs:
  client.create_example(
      inputs={"question": input_prompt},
      outputs={"answer": output_answer},
      dataset_id=dataset.id
  )

Avaliação dos exemplos do dataset

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith.evaluation import evaluate

# Target task definition
prompt = ChatPromptTemplate.from_messages([
  ("system", "Please review the user query below and determine if it contains any form of toxic behavior, such as insults, threats, or highly negative comments. Respond with 'Toxic' if it does, and 'Not toxic' if it doesn't."),
  ("user", "{question}")
])

output_parser = StrOutputParser()

chain = prompt | chat | output_parser

# The name or UUID of the LangSmith dataset to evaluate on.
# Alternatively, you can pass an iterator of examples
data = "Elementary Animal Questions"

# A string to prefix the experiment name with.
# If not provided, a random string will be generated.
experiment_prefix = "Elementary Animal Questions"

# Evaluate the target task
results = evaluate(
  chain.invoke,
  data=data,
  experiment_prefix=experiment_prefix,
)

View the evaluation results for experiment: 'Elementary Animal Questions-04b40433' at:
https://smith.langchain.com/o/15793a46-90bc-5ff7-86c1-16f9efe55fab/datasets/374f26f5-5be9-4381-80eb-d92d16e8365a/compare?selectedSessions=6c1359c3-4d1b-4f23-964b-15a3fd9ecd46




0it [00:00, ?it/s]